In [1]:
#ひとまずライブラリのインポート
import numpy as np
import pandas as pd
import sklearn
#trainをdfに保存
df = pd.read_csv("./Downloads/Titanic/train.csv")
#dfのファイルの長さ、各種欠損値の合計を算出
print(len(df))
print(df.isnull().sum())

891
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


In [2]:
#欠損値を埋める関数
def preprocess(df):
    df['Fare'] =df['Fare'].fillna(df['Fare'].mean())
    df['Age'] =df['Age'].fillna(df['Age'].mean())
    df['Embarked'] = df['Embarked'].fillna(0)
    df = df.drop(['Cabin', 'Name', 'PassengerId', 'Ticket'], axis = 1)
    df['Sex'] = df['Sex'].apply(lambda x: 1 if x == 'male' else 0)
    df['Embarked'][df['Embarked'] == 'S'] = 0
    df['Embarked'][df['Embarked'] == 'C'] = 1
    df['Embarked'][df['Embarked'] == 'Q'] = 2
    return df

In [3]:
#今回ランダムフォレストに必要なライブラリのインポート
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import csv 

/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [6]:
#trainを推測する関数
def train(df):
    #trainデータを学習用とテスト用に分割
    train_x = df.drop('Survived', axis=1)
    train_y = df.Survived
    (train_x, test_x ,train_y, test_y) = train_test_split(train_x, train_y, test_size = 0.33, random_state = 42)
    #randomforest
    clf = RandomForestClassifier(random_state=0)
    clf = clf.fit(train_x, train_y)
    pred = clf.predict(test_x)
    print(accuracy_score(pred, test_y))
    #交差検証
    print(cross_val_score(clf, train_x, train_y, cv = 5))
    
    features = train_x.columns
    importances = clf.feature_importances_
    indices = np.argsort(importances)
    for i in indices[::-1]:
        print("{:<15} {:f}".format(features[i], importances[i]))
    return clf
df = preprocess(df)
clf = train(df)

KeyError: "['Cabin' 'Name' 'PassengerId' 'Ticket'] not found in axis"

In [5]:
df_test_origin = pd.read_csv('./Downloads/Titanic/test.csv')
df_test = preprocess(df_test_origin)
submit_data =  pd.Series(clf.predict(df_test), name='Survived', index=df_test_origin['PassengerId'])
submit_data.to_csv('submit.csv', header=True)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
